In [3]:
import time
import numpy as np
import pylabnet.network.client_server.tektronix_awg70k as tektronix_awg70k
from pylabnet.hardware.awg.tektronix_awg70k import Tek70K
client = tektronix_awg70k.Client(
    host='192.168.50.116',
    port=9565
)
# rm = ResourceManager()
# my_instrument = rm.open_resource('TCPIP0::192.168.50.207::INSTR')
tek = Tek70K(client)

In [2]:
TEK_MAX_AMP = 0.5
TEK_SAMPLE_RATE = 50E9
TEK_PATH = "C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\"
TEK_MAX_CHAR = 800000
TEK_MIN_PULSE_LEN = 4800

In [3]:
def initialize_AWG(tek, sample_rate, awg_max_amp, N_markers):

    ## Reset everything
    tek.reset()
    time.sleep(2)
    tek.wait()
    tek.clear_all_sequences()
    tek.wait()
    tek.clear_all_waveforms()
    tek.wait()

    ## Output settings
    tek.write(f'CLOCK:SRATE {sample_rate}')
    tek.write('OUTPUT1:PATH DIRECT')
    tek.write(f'SOURCE1:VOLTAGE:LEVEL:AMPL {awg_max_amp}')
    # Max 10 bit resolution, 1 bit used per marker, max 2 markers
    tek.write(f'SOURCE1:DAC:RESOLUTION {10-N_markers}')
    # Trigger level
    tek.write('TRIGGER:LEVEL 0.8, ATRIGGER')
    tek.write('TRIGGER:LEVEL 0.8, BTRIGGER')
    tek.wait()


def write_wait_sequence(tek, seq, wait_samp=4800):
    tek.change_plugin("Basic Waveform")
    seq_name, step_ptr = seq.name, seq.step_ptr

    wait_wave_name = "WAIT_WAVE"

    tek.write(f'BWAV:LENGTH {wait_samp}')
    tek.write('BWAV:AMPL 0')
    tek.write(f'BWAV:COMPILE:NAME "{wait_wave_name}"')
    tek.write('BWAV:COMPILE') # Compile
    tek.wait()

    # Add a step
    tek.write(f'SLIST:SEQ:STEP:ADD "{seq_name}", {step_ptr}, 1')
    # Assign the wait waveform to Track 1
    tek.write(f'SLIST:SEQ:STEP{step_ptr}:TASSET1:WAVEFORM "{seq_name}", "{wait_wave_name}"')
    # Make it repeat infinitely
    tek.write(f'SLIST:SEQ:STEP{step_ptr}:RCOUNT "{seq_name}", INF')
    # Add wave table entry
    seq.wave_table[seq.step_ptr] = wait_wave_name

    seq.step_ptr += 1
    return seq

In [40]:
# tek.change_dir("C", "")
tek.delete_file("ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\rabi_waveforms.equ")

In [20]:
tek.write('MMEMORY:CDIRECTORY "\\Users"')

29

In [35]:
tek.get_dir()

'"\\"\n'

In [60]:
def write_rabi_waves(tek, sample_rate, awg_max_amp, pulse_freq, t_rabi, N, min_amp, max_amp):
    tek.change_plugin("Basic Waveform")
    tek.write(f'BWAV:SRATE {sample_rate}') # Sample rate
    tek.write('BWAV:FDR OFF') # Full dynamic range?
    tek.write('BWAV:FUNC SINE') # Waveform Shape
    tek.write(f'BWAV:FREQ {pulse_freq}') # Frequency
    tek.write(f'BWAV:LENGTH {t_rabi * sample_rate}') # Number of samples

    assert(max_amp <= awg_max_amp)
    amps = np.linspace(min_amp, max_amp, N)

    # Generate Rabi waveforms
    for i in range(N):
        tek.write(f'BWAV:AMPL {amps[i]}')
        tek.write(f'BWAV:COMPILE:NAME "Rabi_{i}"') # Name waveform
        tek.write('BWAV:COMPILE') # Compile
        tek.wait()
        tek.write_marker(f"Rabi_{i}", [1 for _ in range(1000)])
        tek.wait()

def write_rabi_waves_equ(tek, pulse_freq, t_rabi, N, min_amp, max_amp, sample_rate):

    equ_filename = "rabi_waveforms.equ"
    equ_filepath = TEK_PATH + equ_filename

    # Deletes any old instance of the file to prevent leftover data
    # Chops the first 3 letters "C:\\" off the path (double backslash is 1 char)
    tek.delete_file(equ_filepath[3:])

    pulse_samples = round(sample_rate * t_rabi)
    equ_str = f"clock = {round(sample_rate)}\n"
    equ_str += f"size = {max(pulse_samples, TEK_MIN_PULSE_LEN)}\n"

    pulse_amps = np.linspace(min_amp, max_amp, N)

    for i in range(N):
        equ_str += f'''"Rabi_{i}" = {pulse_amps[i]} * cos (2 * pi * {pulse_freq} * time) * (time < {t_rabi})\n'''

    tek.write_file(equ_filepath, equ_str)
    size = tek.get_file_size(equ_filepath)
    if size > TEK_MAX_CHAR:
        return 0
    else:
        tek.compile_equation(equ_filepath)
        return 1

def write_rabi_sequence(tek, seq, N, rp):
    seq_name, step_ptr = seq.name, seq.step_ptr

    for rabi_i in range(N):
        for rp_j in range(rp):
            # Add one step to the sequence
            tek.write(f'SLIST:SEQ:STEP:ADD "{seq_name}", {step_ptr}, 1')
            # Set number of repeats = 1
            tek.write(f'SLIST:SEQ:STEP{step_ptr}:RCOUNT "{seq_name}", 1')
            # Assign the Rabi waveform to Track 1
            tek.write(f'SLIST:SEQ:STEP{step_ptr}:TASSET1:WAVEFORM "{seq_name}", "Rabi_{rabi_i}"')
            # Make the wave wait for Trigger A
            tek.write(f'SLIST:SEQ:STEP{step_ptr}:WINPUT "{seq_name}", ATRIGGER')
            # Add wave table entry
            seq.wave_table[step_ptr] = f"Rabi_{rabi_i}"

            step_ptr += 1

    # Make the last step go to first; minus one since step_ptr points to the first empty step
    tek.write(f'SLIST:SEQ:STEP{step_ptr-1}:GOTO "{seq_name}", FIRST')

    seq.step_ptr = step_ptr
    return seq

In [37]:
t_rabi = 10E-6
freq = 100E6
N = 5
min_amp, max_amp = 0.1, 0.5
marker_len = 3000

SAMPLE_RATE = 50E9 # Max 50 GSa/s
AWG_MAX_AMP = 0.5 # 0.3 to 0.5 Vpp
N_MARKERS = 0 # 0, 1 or 2 markers

initialize_AWG(tek, SAMPLE_RATE , AWG_MAX_AMP, N_MARKERS)
write_rabi_waves(tek, SAMPLE_RATE, AWG_MAX_AMP, freq, t_rabi, N, min_amp, max_amp)

seq = tek.create_new_sequence("Hi")
seq = write_rabi_sequence(tek, seq, N, rp=3)
tek.assign_sequence(seq.name)

In [64]:
write_rabi_waves_equ(tek, pulse_freq=10E9, t_rabi=30E-9, N=5, min_amp=0.1, max_amp=0.5, sample_rate=TEK_SAMPLE_RATE)
seq = tek.create_new_sequence("Hi2")
seq = write_rabi_sequence(tek, seq, N=5, rp=1)
tek.assign_sequence(seq.name)

In [5]:
tek.awg_run()
tek.set_output_state(1)
# tek.awg_stop()
# tek.set_output_state(0)

In [44]:
# tek.reset()

In [9]:
# tek.write('MMEM:MSIS "C:"')
# tek.write('MMEM:CDIR "C:\ProgramData\Tektronix\AWG\AWG70k\EquationEditor"')
print(tek.get_dir())
# tek.change_dir("C", "Users")
# print(tek.get_dir())
tek.write('AWGCONTROL:COMP "C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\Untitled.equ"')

"\"



83

In [44]:
tek.write_file("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ",
"""clock = 10e9 'Recommended sample rate
size = 2400
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)""")

In [43]:
tek.write_file("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ", """clock = 10e9 'Recommended sample rate
size = 2400
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)""")

In [45]:
print(tek.read_file("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ"))
print(tek.get_file_size("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ"))

clock = 10e9 'Recommended sample rate
size = 2400
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5))
225


In [71]:
tek.write('WLIST:WAVEFORM:DELETE "Test1"')
tek.write('WLIST:WAVEFORM:NEW "Test1", 4800')

34

In [35]:
import struct
value = 5.1
ba = bytearray(struct.pack("f", value))
print([ "0x%02x" % b for b in ba ])

['0x33', '0x33', '0xa3', '0x40']


In [38]:
struct.pack("<f", value).hex()

'3333a340'

In [99]:
# tek.write_binary_values('WLIST:WAVEFORM:DATA "Test1",0,1500,', np.linspace(0, 1, 1500) , datatype='f', is_big_endian=False)
tek.write_binary_values('WLIST:WAVEFORM:DATA "Test1",0,4800,', 0*np.sin(2 * np.pi * 1E9 * np.arange(0, 4800)/50E9) , datatype='f', is_big_endian=False)


19244

In [ ]:
tek.write('WLISt:WAVeform:OFFSet "Test1",0.')
tek.write('WLISt:WAVeform:SRATE "Test1", 50E9')

In [57]:
tek.write('WLIST:WAVEFORM:NORM "Test1", FSCALE')

37

In [9]:
tek.query('WLIST:WAVEFORM:LMAX?')

32.0